In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.units import cds    
# adding the units to the values           
cds.enable()
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo  
from scipy.interpolate import make_interp_spline

DR16=fits.open("DR16_Z.fits")         #open function reads the file 
FA= Table(DR16[1].data)  
RA=np.array(FA['RA_1'])
DEC=np.array(FA['DEC_1'])
Z=np.array(FA['Z']) 
M=np.array(FA['LOGMBH'])
Me=np.array(FA["LOGMBH_ERR"])
LB=np.array(FA['LOGLBOL'])
Le=np.array(FA["LOGLBOL_ERR"])

In [194]:
def entry(RA,DEC,Z,M,LB,Me,Le):
    data = np.column_stack((RA,DEC,Z,M,LB,Me,Le))
    return(data)

In [196]:
Z_ranges = [(0.0, 1.0), (1.0, 2.0), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8)]
M_ranges = [(3, 7.5), (7.5, 8.5), (8.5, 9.5), (9.5, 11.5)]

def classify (data,Z_ranges,M_ranges):
    subsets = []
    for z in Z_ranges:
        z_subset = data[(data[:,2] > z[0]) & (data[:,2] < z[1])]
        #print(z_subset)
        m_subsets = []
        for m in M_ranges:
            m_subset = z_subset[(z_subset[:,3] > m[0]) & (z_subset[:,3] < m[1])]
            m_subsets.append(m_subset)
        subsets.append(m_subsets)
    return(subsets)

In [250]:
data=entry(RA,DEC,Z,M,LB,Me,Le)
subsets= classify(data,Z_ranges,M_ranges)
subsets[2][1][:,5]

array([0.1411323 , 0.06610274, 0.05825241, ..., 0.09178786, 0.42011649,
       0.24060642])

In [240]:
def submean(subsets,Z_ranges,M_ranges,parameter=4):
    mean=[]
    for i in range(len(Z_ranges)):
        MM=[]
        for j in range(len(M_ranges)):
            MM.append(np.mean(subsets[i][j][:,parameter]))
        mean.append(MM)
    return(mean)
MZ=Mmean(subsets,Z_ranges,M_ranges,parameter=3)
BZ=Mmean(subsets,Z_ranges,M_ranges,parameter=4)
MZe=Mmean(subsets,Z_ranges,M_ranges,parameter=5)
BZe=Mmean(subsets,Z_ranges,M_ranges,parameter=6)

8

In [252]:
from scipy.interpolate import make_interp_spline
def visual(Z_ranges,MZ,BZ,MZe=None, BZe,Zlimit=2.0, grading=300, kernal=3):
    for i in range(len(Z_ranges)):
        if Z_ranges[i][0]< Zlimit:
            pass
        else:
            x = np.array(MZ[i])
            y = np.array(BZ[i])
            x_new = np.linspace(x.min(), x.max(), 300)
            spl = make_interp_spline(x, y, k=3)  # Cubic spline
            y_smooth = spl(x_new)
            plt.plot(x_new, y_smooth, label=f"zbin {Z_ranges[i]}")
            if MZe in not None:
                plt.scatter(MZ[i], BZ[i],xerr=MZe[i], yerr=BZe[i],label=f"zbin {Z_ranges[i]}")
            plt.errorbar(MZ[i], BZ[i],yerr=BZe[i], fmt='o', elinewidth=2, capsize=2, capthick=2, barsabove=True)
            plt.legend()
    plt.xlabel('$log(M_{SMBH} / M_{\odot})$')
    plt.ylabel('$log(L_{Bol.})$')
    plt.rcParams['figure.figsize']=(12,12)
    plt.rcParams.update({'font.size': 12})
    plt.savefig(fname='MLFunction.jpg', transparent=True, bbox_inches="tight")
    return(plt.show())

In [ ]:
evolution=visual(Z_ranges,MZ,BZ,MZe,BZe,Zlimit=2.0, grading=300, kernal=3)